In [21]:
import os
import torch
import cv2
from torchvision import transforms
import numpy as np




In [22]:
# Define the transform
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert NumPy array to PIL Image
    transforms.Resize((112, 112)),  # Resize frame to 112x112
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Function to extract and preprocess frames
def prepare_video(video_path, num_frames=32):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = torch.linspace(0, total_frames - 1, num_frames).long()

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx.item())
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frames.append(transform(frame))
    
    cap.release()

    # Stack frames and format for the model
    video_tensor = torch.stack(frames)  # Shape: [T, C, H, W]
    video_tensor = video_tensor.permute(1, 0, 2, 3)  # [T, C, H, W] -> [C, T, H, W]
    return video_tensor.unsqueeze(0)  # Add batch dimension: [C, T, H, W] -> [B, C, T, H, W]



In [23]:
def process_videos_to_tensor(source_folder, destination_tensor_name, num_frames=32):
    i = 0
    all_videos_tensor = []
    
    # List all files in the source directory
    for filename in os.listdir(source_folder):
        video_path = os.path.join(source_folder, filename)
        i += 1

        if os.path.isfile(video_path):
            print(f"Processing video: {filename}")
            print('Iteration:', i)
            try:
                # Attempt to process the video
                video_tensor = prepare_video(video_path, num_frames)
                all_videos_tensor.append(video_tensor)
                print(f"Successfully processed {filename}")
            except Exception as e:
                # If an error occurs, skip this video and print the error
                print(f"Error processing {filename}: {e}")
                continue  # Skip to the next video

    if all_videos_tensor:
        # Stack all video tensors into a single tensor if there are any processed videos
        all_videos_tensor = torch.cat(all_videos_tensor, dim=0)

        # Save the tensor to a file
        torch.save(all_videos_tensor, destination_tensor_name)
        print(f"All videos have been processed and saved to {destination_tensor_name}")
    else:
        print("No videos were processed successfully.")



In [24]:
process_videos_to_tensor('normal','normal.pt')

Processing video: -WFK_L1GiR8.mp4
Iteration: 1
Successfully processed -WFK_L1GiR8.mp4
Processing video: 1.mp4
Iteration: 2
Successfully processed 1.mp4
Processing video: 10.mp4
Iteration: 3
Successfully processed 10.mp4
Processing video: 100_non_assault.mp4
Iteration: 4
Successfully processed 100_non_assault.mp4
Processing video: 101_non_assault.mp4
Iteration: 5
Successfully processed 101_non_assault.mp4
Processing video: 102_non_assault.mp4
Iteration: 6
Successfully processed 102_non_assault.mp4
Processing video: 103_non_assault.mp4
Iteration: 7
Successfully processed 103_non_assault.mp4
Processing video: 104_non_assault.mp4
Iteration: 8
Successfully processed 104_non_assault.mp4
Processing video: 105_non_assault.mp4
Iteration: 9
Successfully processed 105_non_assault.mp4
Processing video: 106_non_assault.mp4
Iteration: 10
Successfully processed 106_non_assault.mp4
Processing video: 107_non_assault.mp4
Iteration: 11
Successfully processed 107_non_assault.mp4
Processing video: 108_non_

In [8]:
import torch
import cv2
import numpy as np

def save_frame(frame, output_path):
    """
    Saves a single video frame as an image.
    :param frame: The video frame (H, W, C)
    :param output_path: Path to save the output image
    """
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR for OpenCV
    cv2.imwrite(output_path, frame_bgr)  # Save the frame as an image

def main():
    file_path = "video_tensors.pt"
    output_path = "output_frame.jpg"  # Output image file path
    
    # Load the .pt file
    data = torch.load(file_path)
    
    # Extract the first batch (video) from the data
    video_tensor = data[0]  # Shape: (c, t, h, w)
    
    # Rearrange the dimensions to (t, h, w, c) for video playback
    video_tensor = video_tensor.permute(1, 2, 3, 0)  # Shape: (t, h, w, c)
    
    # Convert the tensor to a numpy array
    video_frames = video_tensor.cpu().numpy()
    
    # Convert to 8-bit image format (assuming values are normalized)
    frame = (video_frames[0] * 255).astype(np.uint8)  # Save only the first frame
    
    # Save the first frame as an image
    save_frame(frame, output_path)

if __name__ == "__main__":
    main()


End of video or error in reading frame.
